## 2010 play by play data

In [2]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sn
from sklearn.metrics import silhouette_samples, silhouette_score


sys.setrecursionlimit(100000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

### import 2010-2011 nhl data set

In [111]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_pbp20102011.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_pbp20102011.csv')
da = da.drop('Unnamed: 0', axis=1)

- rename columns and create a season variable

In [112]:
da = da.rename(columns={ 'Game_Id' : 'GameNumber', 'Ev_Zone': 'EventZone', 'Time_Elapsed': 'Time', 'Ev_Team': 'EventTeamCode', 'Description': 'EventDetail', 'Away_Team': 'ATeamCode', 'Home_Team':'HTeamCode', 'p1_name': 'EventPlayer1','p2_name': 'EventPlayer2', 'p3_name': 'EventPlayer3', 'awayPlayer1': 'APlayer1', 'awayPlayer2': 'APlayer2', 'awayPlayer3': 'APlayer3', 'awayPlayer4': 'APlayer4', 'awayPlayer5': 'APlayer5', 'awayPlayer6': 'APlayer6', 'homePlayer1': 'HPlayer1', 'homePlayer2': 'HPlayer2', 'homePlayer3': 'HPlayer3','homePlayer4': 'HPlayer4', 'homePlayer5': 'HPlayer5','homePlayer6': 'HPlayer6', 'Away_Score': 'AGoals', 'Home_Score': 'HGoals', 'Away_Goalie': 'AGoalie', 'Home_Goalie': 'HGoalie', 'Away_Players': 'PlayersA', 'Home_Players': 'PlayersH' })
da['Season'] = 2010
da = da[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'Strength', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer1', 'APlayer2', 'APlayer3', 'APlayer4', 'APlayer5', 'APlayer6', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer1', 'HPlayer2', 'HPlayer3', 'HPlayer4', 'HPlayer5', 'HPlayer6', 'HGoalie', 'PlayersH']]

- keep only regular season games and exclude irrelevant on-ice events

In [113]:
da = da[da['GameNumber'] <= 21230]
da = da[da['Period'] <= 4]
da = da[da['Period'] >= 1]
da = da[da['Event']!='STOP']
da = da[da['Event']!='EISTR']
da = da[da['Event']!='EIEND']
da = da[da['Event']!='PSTR']
da = da[da['Event']!='PEND']
da = da[da['Event']!='SOC']
da = da[da['Event']!='GEND']

- create an event number variable that will count the number of events per game. 
- create an advantage type variable for even strength, power play and short handed situations.

In [114]:
da['EventNumber'] = da.groupby(['Season', 'GameNumber']).cumcount()+1

In [115]:
da['AdvantageType'] = da.apply(lambda x: 'EV' if ((x['PlayersA'] == x['PlayersH']) & (x['PlayersA'] != 1) & (x['PlayersH'] != 1)) else 'PP' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'PP' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'SH' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'SH' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'PP' if ((x['PlayersA'] == 1 ) & (x['PlayersH'] == 1)) else np.nan, axis=1) 

In [116]:
da['AdvantageType'].value_counts()

EV    258601
PP     38064
SH     18082
Name: AdvantageType, dtype: int64

- reshape data from wide to long for away team players and home team players

In [117]:
db = da.copy()
a = [col for col in db.columns if 'APlayer' in col]
b = [col for col in db.columns if 'HPlayer' in col]
db = pd.lreshape(db, {'APlayer' : a, 'HPlayer' : b })
db.head()

,AGoalie,ATeamCode,AdvantageType,Date,Event,EventDetail,EventNumber,EventPlayer1,EventPlayer2,EventPlayer3,EventTeamCode,GameNumber,HGoalie,HTeamCode,Period,PlayersA,PlayersH,Season,Strength,Time,APlayer,HPlayer
0,CAM WARD,CAR,EV,2010-10-07,FAC,MIN won Neu. Zone - CAR #36 JOKINEN vs MIN #9 ...,1,MIKKO KOIVU,JUSSI JOKINEN,NaN,MIN,20003,NIKLAS BACKSTROM,MIN,1,6,6,2010,5x5,0:00,JEFF SKINNER,MIKKO KOIVU
1,CAM WARD,CAR,EV,2010-10-07,FAC,MIN won Neu. Zone - CAR #36 JOKINEN vs MIN #9 ...,2,MIKKO KOIVU,JUSSI JOKINEN,NaN,MIN,20003,NIKLAS BACKSTROM,MIN,1,6,6,2010,5x5,0:08,JEFF SKINNER,MIKKO KOIVU
2,CAM WARD,CAR,EV,2010-10-07,SHOT,"CAR ONGOAL - #12 STAAL, Snap, Off. Zone, 37 ft.",3,ERIC STAAL,NaN,NaN,CAR,20003,NIKLAS BACKSTROM,MIN,1,6,6,2010,5x5,1:05,ERIC STAAL,MATT CULLEN
3,CAM WARD,CAR,EV,2010-10-07,MISS,"CAR #6 GLEASON, Wrist, Wide of Net, Off. Zone,...",4,TIM GLEASON,NaN,NaN,CAR,20003,NIKLAS BACKSTROM,MIN,1,6,6,2010,5x5,1:16,ERIC STAAL,MATT CULLEN
4,CAM WARD,CAR,EV,2010-10-07,HIT,"MIN #22 CLUTTERBUCK HIT CAR #26 COLE, Neu. Zone",5,CAL CLUTTERBUCK,ERIK COLE,NaN,MIN,20003,NIKLAS BACKSTROM,MIN,1,6,6,2010,5x5,1:22,ERIC STAAL,MATT CULLEN


In [118]:
db = db[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'AdvantageType', 'Strength', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer', 'HGoalie', 'PlayersH']]
db = db.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- reshape data from wide to long for away and home team code and player respectfully.

In [119]:
dc = db.copy()
dc = dc.rename(columns={'EventTeamCode': 'EventTeam', 'EventPlayer1': 'EventP1', 'EventPlayer2': 'EventP2', 'EventPlayer3': 'EventP3', 'PlayersA':'PA', 'PlayersH': 'PH'})

In [120]:
a = [col for col in dc.columns if 'Code' in col]
b = [col for col in dc.columns if 'Player' in col]
c = [col for col in dc.columns if 'Goalie' in col]
d = [col for col in dc.columns if 'Position' in col]
dc = pd.lreshape(dc, {'TeamCode' : a, 'PlayerName' : b, 'Goalie': c})

In [121]:
dc = dc.rename(columns={'EventTeam': 'EventTeamCode', 'EventP1': 'EventPlayer1', 'EventP2': 'EventPlayer2', 'EventP3': 'EventPlayer3', 'PA':'PlayersA', 'PH': 'PlayersH'})

In [122]:
dc.shape

(3607391, 19)

### games played

- calculate the games each player participated in for the duration of the 2011 regular season.

In [123]:
s = dc.copy()
s = s[['Season', 'GameNumber', 'TeamCode', 'PlayerName']]
s = s.sort_values(['Season', 'GameNumber', 'TeamCode'], ascending=[True, True, True])
s = s.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [124]:
s['GP'] = s.groupby(['Season', 'TeamCode','PlayerName'])['GameNumber'].transform('count')
s = s.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
s = s[['Season', 'TeamCode', 'PlayerName', 'GP']]
s.head()

,Season,TeamCode,PlayerName,GP
0,2010,MTL,SCOTT GOMEZ,80
1,2010,MTL,BRIAN GIONTA,82
2,2010,MTL,BENOIT POULIOT,79
3,2010,MTL,JOSH GORGES,36
4,2010,MTL,HAL GILL,75


In [125]:
s.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
dtype: int64

In [126]:
s.shape

(1058, 4)

### shots for and against

- calculate the shots against each player faced througout the duration of a season. 
- create shot for and shot against variables. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [127]:
sa = dc.copy()
sa = sa[sa['Event'].isin(['SHOT','GOAL'])]

In [128]:
sa['ShotF'] = sa.apply(lambda x: 1 if x['EventTeamCode'] == x['TeamCode'] else 0, axis=1)
sa['ShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotF'].transform('sum')

sa['ShotA'] = sa.apply(lambda x: 1 if x['EventTeamCode'] != x['TeamCode'] else 0, axis=1)
sa['ShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotA'].transform('sum')

- create shot for and shot against variables for even strength situations only. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [129]:
sa['EVShotF'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
sa['EVShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotF'].transform('sum')

sa['EVShotA'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)
sa['EVShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotA'].transform('sum')

- calculate the shot differential for all shots and even strength shots.

In [130]:
sa['DShots'] = sa['ShotsF'] - sa['ShotsA']
sa['EVDShots'] = sa['EVShotsF'] - sa['EVShotsA']

In [131]:
sa = sa [['Season', 'TeamCode', 'PlayerName', 'ShotsF', 'ShotsA', 'DShots', 'EVShotsF', 'EVShotsA', 'EVDShots' ]]
sa = sa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
sa.head()

,Season,TeamCode,PlayerName,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots
54,2010,MTL,TOMAS PLEKANEC,859,820,39,625,616,9
55,2010,MTL,LARS ELLER,408,406,2,397,395,2
56,2010,MTL,ANDREI KOSTITSYN,725,617,108,559,616,-57
57,2010,MTL,JAROSLAV SPACEK,585,591,-6,509,477,32
58,2010,MTL,PK SUBBAN,978,810,168,746,639,107


In [132]:
sa.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
dtype: int64

In [133]:
sa.shape

(1057, 9)

### goals for and against

- create goals for and goals against variable. If event team code is the same as team code assign a value of 1 for goals for and a value of 0 for goals against. If event team code is different to team code, assign a value of 0 for golas for and a value of 1 for goals against. Display total goals for and against per player for a season.

In [134]:
gfa = dc.copy()
gfa = gfa[gfa['Event'] == 'GOAL']

In [135]:
gfa['EVGoalF'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
gfa['EVGoalA'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)

In [136]:
gfa['GoalF'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] == x['TeamCode']) else 0, axis=1)
gfa['GoalA'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] != x['TeamCode']) else 0, axis=1)


In [137]:
gfa['EVGoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalF'].transform('sum')
gfa['EVGoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalA'].transform('sum')
gfa['GoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalF'].transform('sum')
gfa['GoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalA'].transform('sum')

In [138]:
gfa = gfa[['Season', 'GameNumber', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA']]
gfa = gfa.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
gfa['Plus/Minus'] = gfa['EVGoalsF'] - gfa['EVGoalsA']
gfa = gfa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
gfa = gfa[['Season', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA', 'Plus/Minus']]
gfa.head()

,Season,TeamCode,PlayerName,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus
108,2010,MTL,DUSTIN BOYD,1,9,1,7,-6
110,2010,MTL,MAXIM LAPIERRE,13,19,13,19,-6
112,2010,MTL,MATHIEU DARCHE,41,19,26,18,8
114,2010,MTL,JAROSLAV SPACEK,47,46,40,30,10
116,2010,MTL,PK SUBBAN,71,63,47,52,-5


In [139]:
gfa.shape

(1037, 8)

### create dataframes for all on-ice event types

#### goals dataframe (dg)

In [140]:
dg = dc.copy()
dg = dg[dg['Event'] == 'GOAL']

In [141]:
dg = dg[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dg = dg.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a goal variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [142]:
dg['Goal'] = dg.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dg = dg.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dg['Goals'] = dg.groupby(['Season','TeamCode', 'PlayerName'])['Goal'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [143]:
dg = dg.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dg = dg.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dg = dg[['Season', 'TeamCode', 'PlayerName', 'Goals']]
dg.head()

,Season,TeamCode,PlayerName,Goals
108,2010,MTL,DUSTIN BOYD,1
474,2010,TOR,TIM BRENT,8
546,2010,TOR,PHIL KESSEL,32
1283,2010,TOR,CLARKE MACARTHUR,21
1326,2010,MTL,JEFF HALPERN,11


In [144]:
dg.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Goals         0
dtype: int64

In [145]:
dg.shape

(751, 4)

#### assists dataframe (dast)

In [146]:
dast = dc.copy()
dast = dast[dast['Event'] == 'GOAL']

In [147]:
dast = dast[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer2', 'EventPlayer3']]
dast = dast.rename(columns={'EventPlayer2': 'Assist1Name', 'EventPlayer3': 'Assist2Name', 'EventTeamCode':'TeamCode'})

- reshape assist players 1 and 2 to create one column of assist players

In [148]:
a = [col for col in dast.columns if 'Name' in col]
dast = pd.lreshape(dast, {'PlayerName' : a})

- create an assist variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [149]:
dast['Assist'] = dast.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dast = dast.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dast['Assists'] = dast.groupby(['Season','TeamCode', 'PlayerName'])['Assist'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [150]:
dast = dast.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dast = dast.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dast = dast[['Season', 'TeamCode', 'PlayerName', 'Assists']]
dast.head()

,Season,TeamCode,PlayerName,Assists
0,2010,TOR,DION PHANEUF,22
6,2010,TOR,TYLER BOZAK,17
12,2010,TOR,NIKOLAI KULEMIN,27
18,2010,MTL,MAXIM LAPIERRE,3
24,2010,PHI,DARROLL POWE,10


In [151]:
dast.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Assists       0
dtype: int64

In [152]:
dast.shape

(847, 4)

#### shot dataframe (ds)

In [153]:
ds = dc.copy()
ds = ds[ds['Event'] == 'SHOT']

In [154]:
ds = ds[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
ds = ds.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})
ds.head()

,Season,GameNumber,EventNumber,Event,TeamCode,PlayerName
54,2010,20001,54,SHOT,TOR,COLBY ARMSTRONG
55,2010,20001,54,SHOT,TOR,COLBY ARMSTRONG
56,2010,20001,54,SHOT,TOR,COLBY ARMSTRONG
57,2010,20001,54,SHOT,TOR,COLBY ARMSTRONG
58,2010,20001,54,SHOT,TOR,COLBY ARMSTRONG


- create a shot variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [155]:
ds['Shot'] = ds.apply(lambda x: 1 if (x['Event'] == 'SHOT') else 0, axis=1)
ds = ds.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
ds['Shots'] = ds.groupby(['Season','TeamCode', 'PlayerName'])['Shot'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [156]:
ds = ds.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
ds = ds.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
ds = ds[['Season', 'TeamCode', 'PlayerName', 'Shots']]
ds.head()

,Season,TeamCode,PlayerName,Shots
54,2010,TOR,COLBY ARMSTRONG,61.0
96,2010,MTL,DUSTIN BOYD,6.0
120,2010,TOR,LUKE SCHENN,124.0
144,2010,TOR,TIM BRENT,52.0
162,2010,MTL,RYAN O'BYRNE,3.0


In [157]:
ds.isnull().sum()

Season        0
TeamCode      0
PlayerName    2
Shots         2
dtype: int64

In [158]:
ds.shape

(948, 4)

#### block dataframe (db)

In [159]:
dbl = dc.copy()
dbl = dbl[dbl['Event'] == 'BLOCK']

In [160]:
dbl['BlockTeam'] = dbl.EventDetail.str.split(' ', expand = True)[6]
dbl['BlockTeamCode'] = dbl.BlockTeam.str.split(' ', expand = True)[0]

In [161]:
dbl = dbl[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventPlayer1', 'BlockTeamCode']]
dbl = dbl.rename(columns={'EventPlayer1': 'PlayerName', 'BlockTeamCode':'TeamCode'})

- create a block variable. If event type is a block assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [162]:
dbl['Block'] = dbl.apply(lambda x: 1 if (x['Event'] == 'BLOCK') else 0, axis=1)
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dbl['Blocks'] = dbl.groupby(['Season','TeamCode', 'PlayerName'])['Block'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [163]:
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dbl = dbl.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dbl = dbl[['Season', 'TeamCode', 'PlayerName', 'Blocks']]
dbl.head()

,Season,TeamCode,PlayerName,Blocks
24,2010,MTL,ALEXANDRE PICARD,59
48,2010,TOR,COLBY ARMSTRONG,27
138,2010,TOR,PHIL KESSEL,22
191,2010,MTL,TOMAS PLEKANEC,45
318,2010,TOR,LUKE SCHENN,168


In [164]:
dbl.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Blocks        0
dtype: int64

In [165]:
dbl.shape

(1071, 4)

#### hit dataframe (dh)

In [166]:
dhit = dc.copy()
dhit = dhit[dhit['Event'] == 'HIT']

In [167]:
dhit = dhit[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dhit = dhit.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a hit variable. If event type is a hit assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [168]:
dhit['Hit'] = dhit.apply(lambda x: 1 if (x['Event'] == 'HIT') else 0, axis=1)
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dhit['Hits'] = dhit.groupby(['Season','TeamCode', 'PlayerName'])['Hit'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [169]:
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dhit = dhit.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dhit = dhit[['Season', 'TeamCode', 'PlayerName', 'Hits']]
dhit.head()

,Season,TeamCode,PlayerName,Hits
6,2010,TOR,TIM BRENT,104.0
12,2010,MTL,TOMAS PLEKANEC,53.0
18,2010,MTL,PK SUBBAN,110.0
30,2010,TOR,MIKE BROWN,105.0
66,2010,TOR,FREDRIK SJOSTROM,108.0


In [170]:
dhit.isnull().sum()

Season        0
TeamCode      0
PlayerName    2
Hits          2
dtype: int64

In [171]:
dhit.shape

(976, 4)

#### penalty dataframe (dpen)

In [172]:
dp = dc.copy()
dp = dp[dp['Event'] == 'PENL']

- use event detail to find the duration of a given penalty and to assign it to the proper player. Major penalty is 5 minutes, so a value of 5 is assigned for every event that had a major penalty.

In [173]:
dp['Pen'] = dp.EventDetail.str.split(')', expand = True)[0]
dp['Penal'] = dp.Pen.str.split('(', expand = True)[1]
dp['Penalt'] = dp.Penal.str.split(' ', expand = True)[0]
dp['Penalty'] = dp.apply(lambda x: 5 if x['Penalt'] == 'maj' else x['Penalt'], axis=1)
dp['Penalty'] = dp['Penalty'].convert_objects(convert_numeric=True)
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


- use event detail to find penalty team code and penalty player name.

In [174]:
dp['PenaltyTeamCode'] = dp.EventDetail.str.split('\\s', expand = True)[0]
dp['PN'] = dp.EventDetail.str.split('\\s', expand = True)[1]
dp['PNumber'] = dp.PN.str.split('#', expand = True)[1]
dp['PenaltyPlayerNumber'] = dp.PNumber.str.split(' ', expand = True)[0]
dp['PenaltyPlayerLName'] = dp.EventDetail.str.split('\\s', expand = True)[2]
#dp['PenaltyPlayerLName'] = dp.PPlLN.str.split(' ', expand = True)[0]
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName']]

- seperate player name into first and last. The purpose is to connect the correct player name to penalty player name.

In [175]:
dp['PlayerFName'] = dp.PlayerName.str.split('\\s', expand = True)[0]
dp['PlayerLName'] = dp.PlayerName.str.split('\\s', expand = True)[1]
dp['PenaltyPlayerFName'] = dp.apply(lambda x: x['PlayerFName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName'])) else np.nan, axis=1)
dp['PenaltyPlayerFName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyPlayerFName'].apply(lambda x: x.ffill().bfill())

In [176]:
dp['PenaltyName'] = dp.apply(lambda x: x['PlayerName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName']) & (x['PlayerFName'] == x['PenaltyPlayerFName'])) else np.nan, axis=1)
dp['PenaltyName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerLName', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyName'].apply(lambda x: x.ffill().bfill())

- drop duplicates per game and event number. Calculate the total number of penalties for each player.

In [177]:
dp = dp.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])
dp['Penalties'] = dp.groupby(['Season', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])['Penalty'].transform('sum')
dp['Penalties'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PlayerName'])['Penalties'].apply(lambda x: x.ffill().bfill())
#dp.head()

- drop duplicates to keep one observation per player for the season.

In [178]:
dp = dp[['Season', 'PenaltyTeamCode', 'PenaltyName', 'Penalties']]
dp = dp.drop_duplicates(['Season', 'PenaltyTeamCode', 'PenaltyName'])
dp = dp.rename(columns={'PenaltyTeamCode':'TeamCode', 'PenaltyName':'PlayerName'})
dp = dp[['Season', 'TeamCode', 'PlayerName', 'Penalties']]
dp.head()

,Season,TeamCode,PlayerName,Penalties
175,2010,MTL,LARS ELLER,48.0
281,2010,MTL,MATHIEU DARCHE,10.0
675,2010,TOR,LUKE SCHENN,34.0
970,2010,TOR,DION PHANEUF,88.0
1035,2010,TOR,NaN,NaN


- keep observations that are not null. Since a team can be allocated a penalty, those observations are excluded from the data.

In [179]:
dp = dp[dp['PlayerName'].notnull()]

In [180]:
dp.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Penalties     0
dtype: int64

In [181]:
dp.shape

(911, 4)

####  miss dataframe (dmiss)

In [182]:
dmiss = dc.copy()
dmiss = dmiss[dmiss['Event'] == 'MISS']

In [183]:
dmiss = dmiss[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dmiss = dmiss.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a miss variable. If event type is a miss assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [184]:
dmiss['Miss'] = dmiss.apply(lambda x: 1 if (x['Event'] == 'MISS') else 0, axis=1)
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dmiss['Misses'] = dmiss.groupby(['Season','TeamCode', 'PlayerName'])['Miss'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [185]:
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dmiss = dmiss.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dmiss = dmiss[['Season', 'TeamCode', 'PlayerName', 'Misses']]
dmiss.head()

,Season,TeamCode,PlayerName,Misses
42,2010,MTL,LARS ELLER,48
168,2010,TOR,DION PHANEUF,94
275,2010,TOR,LUKE SCHENN,56
297,2010,TOR,TOMAS KABERLE,44
342,2010,MTL,PK SUBBAN,78


In [186]:
dmiss.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Misses        0
dtype: int64

In [187]:
dmiss.shape

(912, 4)

####  takeaway dataframe (dtake)

In [188]:
dtake = dc.copy()
dtake = dtake[dtake['Event'] == 'TAKE']

In [189]:
dtake = dtake[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dtake = dtake.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a takeaway variable. If event type is a takeaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [190]:
dtake['Takeaway'] = dtake.apply(lambda x: 1 if (x['Event'] == 'TAKE') else 0, axis=1)
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dtake['Takeaways'] = dtake.groupby(['Season','TeamCode', 'PlayerName'])['Takeaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [191]:
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dtake = dtake.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dtake = dtake[['Season', 'TeamCode', 'PlayerName', 'Takeaways']]
dtake.head()

,Season,TeamCode,PlayerName,Takeaways
36,2010,MTL,LARS ELLER,24
201,2010,MTL,JEFF HALPERN,15
206,2010,MTL,JAROSLAV SPACEK,30
269,2010,TOR,MIKHAIL GRABOVSKI,49
302,2010,TOR,KRIS VERSTEEG,40


In [192]:
dtake.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Takeaways     0
dtype: int64

In [193]:
dtake.shape

(870, 4)

####  giveaway dataframe (dgive)

In [194]:
dgive = dc.copy()
dgive = dgive[dgive['Event'] == 'GIVE']

In [195]:
dgive = dgive[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dgive = dgive.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a giveaway variable. If event type is a giveaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [196]:
dgive['Giveaway'] = dgive.apply(lambda x: 1 if (x['Event'] == 'GIVE') else 0, axis=1)
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dgive['Giveaways'] = dgive.groupby(['Season','TeamCode', 'PlayerName'])['Giveaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [197]:
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dgive = dgive.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dgive = dgive[['Season', 'TeamCode', 'PlayerName', 'Giveaways']]
dgive.head()

,Season,TeamCode,PlayerName,Giveaways
60,2010,TOR,JEAN-SEBASTIEN GIGUERE,33
72,2010,MTL,JOSH GORGES,24
102,2010,TOR,CARL GUNNARSSON,44
348,2010,MTL,MATHIEU DARCHE,10
378,2010,TOR,FRANCOIS BEAUCHEMIN,67


In [198]:
dgive.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Giveaways     0
dtype: int64

In [199]:
dgive.shape

(969, 4)

### import 2010 player shifts

In [201]:
dshift = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_shifts20102011.csv')
#dshift = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_shifts20102011.csv')
dshift = dshift.drop('Unnamed: 0', axis=1)

- rename variables, keep only regular season games and create a season variable.

In [202]:
dshift= dshift.rename(columns={ 'Game_Id' : 'GameNumber', 'Team': 'TeamCode', 'Player': 'PlayerName' })
dshift = dshift[dshift['GameNumber'] <= 21230]
dshift['Season'] = 2010
dshift = dshift[['Season', 'Date', 'GameNumber', 'Period', 'TeamCode', 'PlayerName', 'Player_Id', 'Start', 'End', 'Duration']]
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration
0,2010,2010-10-07,20003,1,MIN,MIKKO KOIVU,8469459.0,0.0,47.0,47.0
1,2010,2010-10-07,20003,1,MIN,ANDREW BRUNETTE,8459596.0,0.0,52.0,52.0
2,2010,2010-10-07,20003,1,MIN,ANTTI MIETTINEN,8468704.0,0.0,52.0,52.0
3,2010,2010-10-07,20003,1,MIN,CAM BARKER,8471216.0,0.0,52.0,52.0
4,2010,2010-10-07,20003,1,MIN,GREG ZANON,8468636.0,0.0,52.0,52.0


- calcuate the seconds an individual played per game. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [203]:
dshift['Seconds'] = dshift.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])['Duration'].transform('sum')
dshift['Minutes'] = dshift['Seconds']/60
dshift['Minutes'] = dshift['Minutes'].round(2)

In [204]:
dshift = dshift.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [205]:
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes
0,2010,2010-10-07,20003,1,MIN,MIKKO KOIVU,8469459.0,0.0,47.0,47.0,1269.0,21.15
1,2010,2010-10-07,20003,1,MIN,ANDREW BRUNETTE,8459596.0,0.0,52.0,52.0,1037.0,17.28
2,2010,2010-10-07,20003,1,MIN,ANTTI MIETTINEN,8468704.0,0.0,52.0,52.0,1108.0,18.47
3,2010,2010-10-07,20003,1,MIN,CAM BARKER,8471216.0,0.0,52.0,52.0,1158.0,19.30
4,2010,2010-10-07,20003,1,MIN,GREG ZANON,8468636.0,0.0,52.0,52.0,1346.0,22.43


- calcuate the seconds an individual played for the whole regular season. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [206]:
dshift['TSeconds'] = dshift.groupby(['Season', 'TeamCode', 'PlayerName'])['Seconds'].transform('sum')
dshift['TOI'] = dshift['TSeconds']/60
dshift['TOI'] = dshift['TOI'].round(2)
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes,TSeconds,TOI
0,2010,2010-10-07,20003,1,MIN,MIKKO KOIVU,8469459.0,0.0,47.0,47.0,1269.0,21.15,83033.0,1383.88
1,2010,2010-10-07,20003,1,MIN,ANDREW BRUNETTE,8459596.0,0.0,52.0,52.0,1037.0,17.28,82653.0,1377.55
2,2010,2010-10-07,20003,1,MIN,ANTTI MIETTINEN,8468704.0,0.0,52.0,52.0,1108.0,18.47,74572.0,1242.87
3,2010,2010-10-07,20003,1,MIN,CAM BARKER,8471216.0,0.0,52.0,52.0,1158.0,19.30,51175.0,852.92
4,2010,2010-10-07,20003,1,MIN,GREG ZANON,8468636.0,0.0,52.0,52.0,1346.0,22.43,105993.0,1766.55


- keep only observation per player for the season

In [207]:
dshift = dshift.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dshift = dshift[['Season', 'TeamCode', 'PlayerName', 'TOI']]
dshift.head()

,Season,TeamCode,PlayerName,TOI
0,2010,MIN,MIKKO KOIVU,1383.88
1,2010,MIN,ANDREW BRUNETTE,1377.55
2,2010,MIN,ANTTI MIETTINEN,1242.87
3,2010,MIN,CAM BARKER,852.92
4,2010,MIN,GREG ZANON,1766.55


In [208]:
dshift.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
TOI           0
dtype: int64

In [209]:
dshift.shape

(1058, 4)

### import 2010 player positions

In [242]:
dpos = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_positions20102011.csv')
#dpos = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_positions20102011.csv')

In [243]:
dpos= dpos.rename(columns={ 'Team': 'TeamCode', 'Old Team' : 'OldTeam', 'Pos ': 'Position'})
dpos = dpos.drop(dpos.columns[dpos.columns.str.contains('unnamed',case = False)],axis = 1)
dpos.head()

,Name,TeamCode,Position,PlayerName
0,Justin Abdelkader,DET,L,JUSTIN ABDELKADER
1,Luke Adam,BUF,L,LUKE ADAM
2,Craig Adams,PIT,R,CRAIG ADAMS
3,Andrew Alberts,VAN,D,ANDREW ALBERTS
4,Daniel Alfredsson,OTT,R,DANIEL ALFREDSSON


In [244]:
dpos= dpos.rename(columns={ 'Team': 'TeamCode', 'Pos ': 'Position'})
dpos['Season'] = 2010
dpos = dpos[['Season', 'TeamCode', 'PlayerName', 'Position']]
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2010,DET,JUSTIN ABDELKADER,L
1,2010,BUF,LUKE ADAM,L
2,2010,PIT,CRAIG ADAMS,R
3,2010,VAN,ANDREW ALBERTS,D
4,2010,OTT,DANIEL ALFREDSSON,R


In [245]:
dpos.dtypes

Season         int64
TeamCode      object
PlayerName    object
Position      object
dtype: object

In [246]:
dpos['Position'] = dpos.apply(lambda x: 'W' if x['Position'] == 'L' else 'W' if x['Position'] == 'R' else x['Position'] , axis=1)
dpos['Position'].value_counts()

W    336
D    303
C    252
G     87
Name: Position, dtype: int64

In [247]:
dpos['TeamCode'] = dpos.apply(lambda x: 'L.A' if x['TeamCode'] == 'LAK' else 'N.J' if x['TeamCode'] == 'NJD' else 'S.J' if x['TeamCode'] == 'SJS' else 'T.B' if x['TeamCode'] == 'TBL' else x['TeamCode'] , axis=1)
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2010,DET,JUSTIN ABDELKADER,W
1,2010,BUF,LUKE ADAM,W
2,2010,PIT,CRAIG ADAMS,W
3,2010,VAN,ANDREW ALBERTS,D
4,2010,OTT,DANIEL ALFREDSSON,W


In [248]:
dpos.shape

(978, 4)

### merge data frames

- merge games played, goals and assists.

In [249]:
dd = pd.merge(s, dpos, on=['Season', 'TeamCode', 'PlayerName'], how='left').merge(dg,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dast,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')


- forward and backward fill players that have been traded during the season. Fill in positions for three players.

In [250]:
dd['Position'] = dd.groupby(['PlayerName'])['Position'].apply(lambda x: x.ffill().bfill())
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists
0,2010,MTL,SCOTT GOMEZ,80,C,7.0,31.0
1,2010,MTL,BRIAN GIONTA,82,W,29.0,17.0
2,2010,MTL,BENOIT POULIOT,79,W,13.0,17.0
3,2010,MTL,JOSH GORGES,36,D,1.0,6.0
4,2010,MTL,HAL GILL,75,D,2.0,7.0


In [251]:
dd.isnull().sum()

Season          0
TeamCode        0
PlayerName      0
GP              0
Position        0
Goals         307
Assists       211
dtype: int64

- fill in goals and assists with 0 for players who had no points.

In [253]:
dd['Goals'] = dd['Goals'].fillna(0)
dd['Assists'] = dd['Assists'].fillna(0)
dd['Points'] = dd['Goals'] + dd['Assists']
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points
0,2010,MTL,SCOTT GOMEZ,80,C,7.0,31.0,38.0
1,2010,MTL,BRIAN GIONTA,82,W,29.0,17.0,46.0
2,2010,MTL,BENOIT POULIOT,79,W,13.0,17.0,30.0
3,2010,MTL,JOSH GORGES,36,D,1.0,6.0,7.0
4,2010,MTL,HAL GILL,75,D,2.0,7.0,9.0


In [254]:
dd.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
dtype: int64

In [255]:
de = pd.merge(dd, gfa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(ds,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(sa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dbl,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dhit,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dp,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dtake,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dgive,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dmiss,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dshift,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')
de.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus,Shots,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots,Blocks,Hits,Penalties,Takeaways,Giveaways,Misses,TOI
0,2010,MTL,SCOTT GOMEZ,80,C,7.0,31.0,38.0,58.0,52.0,34.0,47.0,-13.0,150.0,898.0,693.0,205.0,712.0,638.0,74.0,23.0,32.0,48.0,56.0,28.0,57.0,1485.08
1,2010,MTL,BRIAN GIONTA,82,W,29.0,17.0,46.0,74.0,55.0,49.0,43.0,6.0,269.0,963.0,739.0,224.0,745.0,618.0,127.0,30.0,87.0,22.0,45.0,27.0,111.0,1608.75
2,2010,MTL,BENOIT POULIOT,79,W,13.0,17.0,30.0,40.0,33.0,35.0,33.0,2.0,117.0,523.0,415.0,108.0,465.0,413.0,52.0,20.0,110.0,87.0,26.0,14.0,43.0,911.32
3,2010,MTL,JOSH GORGES,36,D,1.0,6.0,7.0,25.0,35.0,23.0,25.0,-2.0,19.0,297.0,414.0,-117.0,284.0,306.0,-22.0,79.0,55.0,18.0,8.0,24.0,19.0,762.23
4,2010,MTL,HAL GILL,75,D,2.0,7.0,9.0,41.0,69.0,41.0,50.0,-9.0,60.0,611.0,836.0,-225.0,607.0,593.0,14.0,150.0,75.0,43.0,21.0,62.0,29.0,1486.48


- fill in with 0 for all nan values since not all players have been on the ice for a goal for or a goal against. 

In [259]:
de['GoalsF'] = de['GoalsF'].fillna(0)
de['GoalsA'] = de['GoalsA'].fillna(0)
de['EVGoalsF'] = de['EVGoalsF'].fillna(0)
de['EVGoalsA'] = de['EVGoalsA'].fillna(0)
de['ShotsF'] = de['ShotsF'].fillna(0)
de['ShotsA'] = de['ShotsA'].fillna(0)
de['DShots'] = de['DShots'].fillna(0)
de['EVShotsF'] = de['EVShotsF'].fillna(0)
de['EVShotsA'] = de['EVShotsA'].fillna(0)
de['EVDShots'] = de['EVDShots'].fillna(0)
de['Plus/Minus'] = de['Plus/Minus'].fillna(0)
de['Shots'] = de['Shots'].fillna(0)
de['Hits'] = de['Hits'].fillna(0)
de['Blocks'] = de['Blocks'].fillna(0)
de['Penalties'] = de['Penalties'].fillna(0)
de['Giveaways'] = de['Giveaways'].fillna(0)
de['Takeaways'] = de['Takeaways'].fillna(0)
de['Misses'] = de['Misses'].fillna(0)

In [260]:
de.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
GoalsF        0
GoalsA        0
EVGoalsF      0
EVGoalsA      0
Plus/Minus    0
Shots         0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
Blocks        0
Hits          0
Penalties     0
Takeaways     0
Giveaways     0
Misses        0
TOI           0
dtype: int64

### total player stats for the season

- - players that have been traded need to be evaluated for their season performance. Group by sesaon and player name to calcute the sum of each stat. 

In [261]:
de['TGP'] = de.groupby(['Season', 'PlayerName'])['GP'].transform('sum')
de['TGoals'] = de.groupby(['Season', 'PlayerName'])['Goals'].transform('sum')
de['TAssists'] = de.groupby(['Season', 'PlayerName'])['Assists'].transform('sum')
de['TPoints'] = de.groupby(['Season', 'PlayerName'])['Points'].transform('sum')
de['TGoalsF'] = de.groupby(['Season', 'PlayerName'])['GoalsF'].transform('sum')
de['TGoalsA'] = de.groupby(['Season', 'PlayerName'])['GoalsA'].transform('sum')
de['TEVGoalsF'] = de.groupby(['Season', 'PlayerName'])['EVGoalsF'].transform('sum')
de['TEVGoalsA'] = de.groupby(['Season', 'PlayerName'])['EVGoalsA'].transform('sum')
de['TPlus/Minus'] = de.groupby(['Season', 'PlayerName'])['Plus/Minus'].transform('sum')
de['TShots'] = de.groupby(['Season', 'PlayerName'])['Shots'].transform('sum')
de['TShotsF'] = de.groupby(['Season', 'PlayerName'])['ShotsF'].transform('sum')
de['TShotsA'] = de.groupby(['Season', 'PlayerName'])['ShotsA'].transform('sum')
de['TDShots'] = de.groupby(['Season', 'PlayerName'])['DShots'].transform('sum')
de['TEVShotsF'] = de.groupby(['Season', 'PlayerName'])['EVShotsF'].transform('sum')
de['TEVShotsA'] = de.groupby(['Season', 'PlayerName'])['EVShotsA'].transform('sum')
de['TEVDShots'] = de.groupby(['Season', 'PlayerName'])['EVDShots'].transform('sum')
de['TBlocks'] = de.groupby(['Season', 'PlayerName'])['Blocks'].transform('sum')
de['THits'] = de.groupby(['Season', 'PlayerName'])['Hits'].transform('sum')
de['TPenalties'] = de.groupby(['Season', 'PlayerName'])['Penalties'].transform('sum')
de['TTakeaways'] = de.groupby(['Season', 'PlayerName'])['Takeaways'].transform('sum')
de['TGiveaways'] = de.groupby(['Season', 'PlayerName'])['Giveaways'].transform('sum')
de['TMisses'] = de.groupby(['Season', 'PlayerName'])['Misses'].transform('sum')
de['TTOI'] = de.groupby(['Season', 'PlayerName'])['TOI'].transform('sum')

- divide all stats with total time on ice to scale the data for cluster analysis

In [262]:
de['TOIGoals'] = de['TGoals'] / de['TTOI']
de['TOIAssists'] = de['TAssists'] / de['TTOI']
de['TOIPoints'] = de['TPoints'] / de['TTOI']
de['TOIGoalsF'] = de['TGoalsF'] / de['TTOI']
de['TOIGoalsA'] = de['TGoalsA'] / de['TTOI']
de['TOIEVGoalsF'] = de['TEVGoalsF'] / de['TTOI']
de['TOIEVGoalsA'] = de['TEVGoalsA'] / de['TTOI']
de['TOIPlus/Minus'] = de['TPlus/Minus'] / de['TTOI']
de['TOIShots'] = de['TShots'] / de['TTOI']
de['TOIShotsF'] = de['TShotsF'] / de['TTOI']
de['TOIShotsA'] = de['TShotsA'] / de['TTOI']
de['TOIDShots'] = de['TDShots'] / de['TTOI']
de['TOIEVShotsF'] = de['TEVShotsF'] / de['TTOI']
de['TOIEVShotsA'] = de['TEVShotsA'] / de['TTOI']
de['TOIEVDShots'] = de['TEVDShots'] / de['TTOI']
de['TOIBlocks'] = de['TBlocks'] / de['TTOI']
de['TOIHits'] = de['THits'] / de['TTOI']
de['TOIPenalties'] = de['TPenalties'] / de['TTOI']
de['TOITakeaways'] = de['TTakeaways'] / de['TTOI']
de['TOIGiveaways'] = de['TGiveaways'] / de['TTOI']
de['TOIMisses'] = de['TMisses'] / de['TTOI']

- keep season total variables and scaled variables

In [263]:
de = de[['Season', 'TeamCode', 'PlayerName', 'Position', 'TGP', 'TGoals', 'TAssists', 'TPoints', 'TGoalsF', 'TGoalsA', 'TEVGoalsF', 'TEVGoalsA', 'TPlus/Minus', 'TShots', 'TShotsF', 'TShotsA', 'TDShots', 'TEVShotsF', 'TEVShotsA', 'TEVDShots', 'TBlocks', 'THits', 'TPenalties', 'TTakeaways', 'TGiveaways', 'TMisses', 'TTOI', 'TOIGoals', 'TOIAssists', 'TOIPoints', 'TOIGoalsF', 'TOIGoalsA', 'TOIEVGoalsF', 'TOIEVGoalsA', 'TOIPlus/Minus', 'TOIShots', 'TOIShotsF', 'TOIShotsA', 'TOIDShots', 'TOIEVShotsF', 'TOIEVShotsA', 'TOIEVDShots', 'TOIBlocks', 'TOIHits', 'TOIPenalties', 'TOITakeaways', 'TOIGiveaways', 'TOIMisses']]
de.head()

,Season,TeamCode,PlayerName,Position,TGP,TGoals,TAssists,TPoints,TGoalsF,TGoalsA,TEVGoalsF,TEVGoalsA,TPlus/Minus,TShots,TShotsF,TShotsA,TDShots,TEVShotsF,TEVShotsA,TEVDShots,TBlocks,THits,TPenalties,TTakeaways,TGiveaways,TMisses,TTOI,TOIGoals,TOIAssists,TOIPoints,TOIGoalsF,TOIGoalsA,TOIEVGoalsF,TOIEVGoalsA,TOIPlus/Minus,TOIShots,TOIShotsF,TOIShotsA,TOIDShots,TOIEVShotsF,TOIEVShotsA,TOIEVDShots,TOIBlocks,TOIHits,TOIPenalties,TOITakeaways,TOIGiveaways,TOIMisses
0,2010,MTL,SCOTT GOMEZ,C,80,7.0,31.0,38.0,58.0,52.0,34.0,47.0,-13.0,150.0,898.0,693.0,205.0,712.0,638.0,74.0,23.0,32.0,48.0,56.0,28.0,57.0,1485.08,0.004714,0.020874,0.025588,0.039055,0.035015,0.022894,0.031648,-0.008754,0.101005,0.604681,0.466642,0.138040,0.479435,0.429606,0.049829,0.015487,0.021548,0.032321,0.037708,0.018854,0.038382
1,2010,MTL,BRIAN GIONTA,W,82,29.0,17.0,46.0,74.0,55.0,49.0,43.0,6.0,269.0,963.0,739.0,224.0,745.0,618.0,127.0,30.0,87.0,22.0,45.0,27.0,111.0,1608.75,0.018026,0.010567,0.028594,0.045998,0.034188,0.030458,0.026729,0.003730,0.167211,0.598601,0.459363,0.139239,0.463092,0.384149,0.078943,0.018648,0.054079,0.013675,0.027972,0.016783,0.068998
2,2010,MTL,BENOIT POULIOT,W,79,13.0,17.0,30.0,40.0,33.0,35.0,33.0,2.0,117.0,523.0,415.0,108.0,465.0,413.0,52.0,20.0,110.0,87.0,26.0,14.0,43.0,911.32,0.014265,0.018654,0.032919,0.043892,0.036211,0.038406,0.036211,0.002195,0.128385,0.573893,0.455383,0.118509,0.510249,0.453189,0.057060,0.021946,0.120704,0.095466,0.028530,0.015362,0.047184
3,2010,MTL,JOSH GORGES,D,36,1.0,6.0,7.0,25.0,35.0,23.0,25.0,-2.0,19.0,297.0,414.0,-117.0,284.0,306.0,-22.0,79.0,55.0,18.0,8.0,24.0,19.0,762.23,0.001312,0.007872,0.009184,0.032798,0.045918,0.030175,0.032798,-0.002624,0.024927,0.389646,0.543143,-0.153497,0.372591,0.401454,-0.028863,0.103643,0.072157,0.023615,0.010496,0.031487,0.024927
4,2010,MTL,HAL GILL,D,75,2.0,7.0,9.0,41.0,69.0,41.0,50.0,-9.0,60.0,611.0,836.0,-225.0,607.0,593.0,14.0,150.0,75.0,43.0,21.0,62.0,29.0,1486.48,0.001345,0.004709,0.006055,0.027582,0.046418,0.027582,0.033637,-0.006055,0.040364,0.411038,0.562402,-0.151364,0.408347,0.398929,0.009418,0.100910,0.050455,0.028927,0.014127,0.041709,0.019509


In [264]:
de.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/stats/2010_player_stats.csv', index='False', sep=',')
#de = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/stats/2010_player_stats.csv', index='False', sep=',')